## IMPORTS

In [1]:
# Twitter Sentiment Analysis Imports
import os
import tweepy
import textblob
import re
import requests, json
from datetime import datetime
from tweepy import OAuthHandler
from textblob import TextBlob
%load_ext dotenv
%dotenv

# Bollinger Band Imports
import krakenex
import pandas as pd
import numpy as np
from pykrakenapi import KrakenAPI

# User Interface Imports
import tkinter as tk

## 1) SENTIMENT ANALYSIS FOR BITCOIN (USING TWITTER API)

In [2]:
# Code ref: https://medium.com/@BlockchainEng/crypto-trading-bot-sentiment-analysis-bot-bfbd8dd1df5a

In [3]:
# Sourced Twitter Sentiment Analysis Algorithm that we repurposed for Bitcoin
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analconsumer_key
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = os.environ['TWITTER_CONSUMER_KEY']
        consumer_secret = os.environ['TWITTER_CONSUMER_SECRET_KEY']
        access_token = os.environ['TWITTER_TOKEN']
        access_token_secret = os.environ['TWITTER_TOKEN_SECRET_KEY']
       
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def tweet_file(self):
        self.message="Sentiment Analysis by @BlockchainEng"
        self.filename='SentimentAnalysis.png'
        print(self)
        self.api.update_with_media(message = self.message, filename= self.filename)

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # parsing tweets one by one
            list_of_tweets = []
            for tweet in fetched_tweets:

                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                #tweet1 =
                #tweet1=TwitterClient.api.clean_tweet(tweet_text)
                parsed_tweet['text'] = tweet.text
                list_of_tweets.append(tweet.text)
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            with open('TweetHistory.txt', 'a+') as f1:
                for tweet in list_of_tweets:
                    try:
                        f1.write(tweet)
                    except:
                        pass

            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    search_term = 'Bitcoin'
    list_coins=['Bitcoin'] #, 'Ethereum', 'EOS', 'Cryptocurrency', 'Blockchain', 'BTC'] #'Donald Trump', 'Barack Obama', 'George Washington']
    count1=200
    list_coin_val = []
    start_time = datetime.now()
    try:
        for i in range(0,1):
            #This For Loop controls how many data 'cycles' are collected before visualization and tweeting
            for coin in list_coins:
                tweets = api.get_tweets(query = coin, count = count1)

                # picking positive tweets from tweets
                ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
                # percentage of positive tweets
                print("\n\nTime: {}".format(str(datetime.now())))
                print("Sentiment Values for {}\nNumber of Tweets Analyzed: {}".format(coin, count1))
                positive_tweet_percentage = round(100*len(ptweets)/len(tweets),6)
                print("Positive tweets percentage: {} %".format(positive_tweet_percentage))
                # picking negative tweets from tweets
                ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
                # percentage of negative tweets
                negative_tweet_percentage = round(100*len(ntweets)/len(tweets),6)
                print("Negative tweets percentage: {} %".format(negative_tweet_percentage))
        return positive_tweet_percentage, negative_tweet_percentage
    except:
        print("ERROR - COLLECTING DATA")

In [4]:
# Printing sample of our Twitter sentiment analysis for Bitcoin
positive_tweet_percentage, negative_tweet_percentage = main()



Time: 2021-06-19 22:50:06.549485
Sentiment Values for Bitcoin
Number of Tweets Analyzed: 200
Positive tweets percentage: 40.963855 %
Negative tweets percentage: 12.048193 %


#### [Extra] Saving sentiment and price data into CSV for our own external analysis [Extra]

In [5]:
# Saving historical sentiment analysis data for our own analysis.
def save_to_file(coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, time, current_price, filename = "SentimentHistorical.csv"):
    with open(filename, 'a+') as  f:
        line = coin, positive_tweet_percentage, negative_tweet_percentage, neutral_tweet_percentage, time, current_price
        f.writelines(str(line) + '\n')
        
# Saving historical price data for our own analysis.
def getBitcoinPrice():
    URL = 'https://www.bitstamp.net/api/ticker/'
    try:
        r = requests.get(URL)
        priceFloat = float(json.loads(r.text)['last'])
        return priceFloat
    except requests.ConnectionError:
        print("Error querying Bitstamp API")
        
def tweet_file(self, message, filename):
    self.api.update_with_media(message, filename)
    
def historic_data_viz(self):
    #Load Historic Data and Visual into graph form for entire recorded amount
    historic_data_list = []
    i=0
    with open('SentimentHistorical.csv') as f1:
        lines = list(f1.readlines())
        for line in lines:
            print(line)
            data=list(line.split(','))
            print(data)
            if i==0:
                start_time=data[5]
            if i==len(f1.readlines()):
                end_time=data[5]
            historic_data_list.append(data)
            i+=1
    data_visualize(historic_data_list, start_time, end_time, self)

def trading(current_price, positive_sentiment_percent, negative_sentiment_percent):
    print("Positive Sentiment - Negative Sentiment (Net Positive)", positive_sentiment_percent-negative_sentiment_percent)
    if positive_sentiment_percent>1.5*negative_sentiment_percent:
        #Buy
        #use Khal's Code to place order, entries & exits
        print("TEST - BUY SIGNAL")
        pass
    if negative_sentiment_percent>1.5*positive_sentiment_percent:
        #Sell
        print("TEST - SELL SIGNAL")
        #use Khal's Code to place order, entries & exits
        pass

#### Creating Buy/Sell Signals using Sentiment Analysis

In [6]:
# Defining Get Sentiment Signal Function
def get_sentiment_signal():
    positive_tweet_percentage, negative_tweet_percentage = main()
    if positive_tweet_percentage > 50:
        sentiment_signal = 1
    elif negative_tweet_percentage > 50:
        sentiment_signal = -1
    else: sentiment_signal = 0
    return sentiment_signal

## 2) BOLLINGER BAND ANALYSIS FOR BITCOING (USING KRAKEN API)

In [7]:
# Code Ref: https://github.com/IgorWounds/Kraken-API---A-Complete-Guide-Algotrading101/blob/main/Kraken%20Public%20Endpoints.ipynb

#### Preparing Data for Bollinger Band Analysis

In [8]:
# Create Bitcoin Open High Low Close "ohlc" Dataframe using Kraken for Bollinger Band Analysis
api = krakenex.API()
k = KrakenAPI(api)
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
ohlc[0]['close'] = ohlc[0]['close'].dropna()
ohlc[0].tail()

,time,open,high,low,close,vwap,volume,count
dtime,,,,,,,,
2021-06-16,1623801600,40167.3,40493.0,38120.0,38337.1,39141.7,6487.206888,49793
2021-06-17,1623888000,38337.1,39561.4,37405.0,38078.2,38433.4,6003.220618,47838
2021-06-18,1623974400,38078.2,38193.1,35126.0,35824.0,36734.4,6558.468890,48363
2021-06-19,1624060800,35824.0,36450.0,34901.0,35490.5,35743.9,4485.076656,39726
2021-06-20,1624147200,35484.2,35792.0,35150.0,35367.5,35468.2,729.910870,3668


#### Creating Buy/Sell Signals using Bollinger Band Analysis

In [9]:
# Defining OHLC update function
def ohlc_update():
    ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
    ohlc[0]['close'] = ohlc[0]['close'].dropna()
    # Set bollinger band window
    bollinger_window = 20

    # Calculate rolling mean and standard deviation
    ohlc[0]['bollinger_mid_band'] = ohlc[0]['close'].rolling(window=bollinger_window).mean()
    ohlc[0]['bollinger_std'] = ohlc[0]['close'].rolling(window=20).std()

    # Calculate upper and lowers bands of bollinger band
    ohlc[0]['bollinger_upper_band']  = ohlc[0]['bollinger_mid_band'] + (ohlc[0]['bollinger_std'] * 1)
    ohlc[0]['bollinger_lower_band']  = ohlc[0]['bollinger_mid_band'] - (ohlc[0]['bollinger_std'] * 1)

    # Calculate bollinger band trading signal
    ohlc[0]['bollinger_long'] = np.where(ohlc[0]['close'] < ohlc[0]['bollinger_lower_band'], 1.0, 0.0)
    ohlc[0]['bollinger_short'] = np.where(ohlc[0]['close'] > ohlc[0]['bollinger_upper_band'], -1.0, 0.0)
    ohlc[0]['bollinger_signal'] = ohlc[0]['bollinger_long'] + ohlc[0]['bollinger_short'] 
    return ohlc[0]['bollinger_signal'][-1]

## 3) CREATING BITCOIN GENIE'S DECISION-MAKING LOGIC

In [10]:
# Defining Get Genie Decision function
def get_genie_decision(bollinger_signal, sentiment_signal):
    if bollinger_signal == 1 and sentiment_signal == 1:
        genie_decision = "It's a good time to BUY! Why? Because BITCOIN is UNDERVALUED and TWITTER is BULLISH on Bitcoin!"
    elif bollinger_signal == 1 and sentiment_signal == 0:
        genie_decision = "It's a good time to BUY! Why? Because BITCOIN is UNDERVALUED and TWITTER IS NEUTRAL ON BITCOIN!"
    elif bollinger_signal == 1 and sentiment_signal == -1:
        genie_decision = "It's a good time to SELL! Why? Because EVEN THOUGH BITCOIN IS UNDERVALUED, TWITTER IS BEARISH ON BITCOIN!"
    elif bollinger_signal == 0 and sentiment_signal == 1:
        genie_decision = "It's a good time to BUY! Why? Because EVEN THOUGH BITCOIN IS NOT UNDERVALUED, TWITTER IS BULLISH ON BITCOIN!"
    elif bollinger_signal == 0 and sentiment_signal == 0:
        genie_decision = "It's a good time to HODL! Why? Because Bitcoin is neither overvalued nor undervalued and Twitter is neutral on Bitcoin!"
    elif bollinger_signal == 0 and sentiment_signal == -1:
        genie_decision = "It's a good time to SELL! Why? Because EVEN THOUGH BITCOIN IS NOT OVERVALUED, TWITTER IS BEARISH ON BITCOIN!"
    elif bollinger_signal == -1 and sentiment_signal == 1:
        genie_decision = "It's a good time to BUY! Why? Because EVEN THOUGH BITCOIN IS OVERVALUED, TWITTER IS BULLISH ON BITCOIN!"
    elif bollinger_signal == -1 and sentiment_signal == 0:
        genie_decision = "It's a good time to SELL! Why? Because BITCOIN IS OVERVALUED AND TWITTER IS NEUTRAL ON BITCOIN!"
    else: #bollinger_signal == -1 and sentiment_signal == -1
        genie_decision = "It's a good time to SELL! Why? Because BITCOIN IS OVERVALUED AND TWITTER IS BEARISH ON BITCOIN!"
    return genie_decision

In [11]:
# Defining Ask Genie Function
def ask_genie():
    bollinger_signal = ohlc_update()
    sentiment_signal = get_sentiment_signal()
    return get_genie_decision(bollinger_signal, sentiment_signal)

## 4) CREATING BITCOIN GENIE'S USER INTERFACE (USING TKINTER)

In [12]:
root = tk.Tk()
root.geometry('500x500')
root.title("Bitcoin Genie")

label_0 = tk.Label(root, text="Bitcoin Genie",width=20,font=("bold", 36))
label_0.place(x=-40,y=53)

label_1 = tk.Label(root, text="“I can tell you whether to buy, hold, or sell Bitcoin at this moment”",width=60,font=("bold", 12))
label_1.place(x=-20,y=110)

label_2 = tk.Label(root, text="DISCLAIMER: THIS IS NOT FINANCIAL ADVICE!",width=60,font=("bold", 12))
label_2.place(x=-20,y=450)

def button_press():
    label_response = tk.Text(root, height=3, width=50)
    label_response.pack()
    #label_response.insert(tk.END,"")
    label_response.config(state="disabled")
    label_response.place(x=50,y=250)
    response = ask_genie()
    label_response.config(state="normal")
    label_response.insert(tk.END,ask_genie())
    label_response.config(state="disabled")

b1=tk.Button(root,text="Ask ME!",font=("bold", 20),height=2,width=10,bg='blue',fg='white', command=button_press)
b1.place(x=170,y=150)

tk.mainloop()



Time: 2021-06-19 22:50:10.524252
Sentiment Values for Bitcoin
Number of Tweets Analyzed: 200
Positive tweets percentage: 39.534884 %
Negative tweets percentage: 11.627907 %


Time: 2021-06-19 22:50:11.594571
Sentiment Values for Bitcoin
Number of Tweets Analyzed: 200
Positive tweets percentage: 37.209302 %
Negative tweets percentage: 10.465116 %
